# When Train/Test Split and Cross Validation can go Wrong

https://www.notion.so/Reminder-Data-Leakage-Attension-dd59e3d7636e415c93a9197dd0068322

> In practice, a random split like we've used here might not be a good idea -- here's what Dr Rachel Thomas has to say about it: "One of the most likely culprits for this disconnect between results in development vs results in production is a poorly chosen validation set (or even worse, no validation set at all). Depending on the nature of your data, choosing a validation set can be the most important step. Although sklearn offers a train_test_split method, this method takes a random subset of the data, which is a poor choice for many real-world problems."
-> https://www.fast.ai/posts/2017-11-13-validation-sets.html

Why? because Random splitting into train and validation data: -> -> https://www.fast.ai/posts/2017-11-13-validation-sets.html
1. Might not be useful in some situation eg. time series prediction task.
2. Lead to Data leakage, and your model ends up doing very well on validation set instead of test data.
  - https://www.alfredo.motta.name/cross-validation-done-wrong/
  - Data Leakage when oversampling and train & test split:
    - https://beckernick.github.io/oversampling-modeling/
  - data in test set never appears inside the train set. So when construct the validation set, we need to mimic that situation. if we not do this, then its likely we overfit: https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/discussion/315220

3. Might not performed the right way with cross validation (didnt include all training (feature selection, preprocessing, etc.) in each fold.)
   - https://gregpark.io/blog/Kaggle-Psychopathy-Postmortem
4. Preprocessing the entire dataset at once: for example, fitting a scaler or PCA on all your data (including test folds) before doing cross-validation splits -> information from outside the training set “bleeds” into the training 


## 2.3 Data Leakage - Group

See k_fold_cv_grouped: 


Data in test set never appears inside the train set. So when construct the validation set, we need to mimic that situation. if we not do this, then its likely we overfit: https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/discussion/315220

Apologies in advance if I'm missing something, but I just checked the validation set code you provide in the notebook, and found that it doesn't actually split the different anchors into different groups. Therefore, I've provided an alternate (and IMO easier to understand) approach here: https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster/

```code
train dataset
id,anchor,target,context,score
37d61fd2272659b1,abatement,abatement of pollution,A47,0.5
7b9652b17b68b7a4,abatement,act of abating,A47,0.75
36d72442aefd8232,abatement,active catalyst,A47,0.25
5296b0c19e1ce60e,abatement,eliminating process,A47,0.5
54c1e3b9184cb5b6,abatement,forest region,A47,0
067203128142739c,abatement,greenhouse gases,A47,0.25
061d17f04be2d1cf,abatement,increased rate,A47,0.25
e1f44e48399a2027,abatement,measurement level,A47,0.25
0a425937a3e86d10,abatement,minimising sounds,A47,0.5
```

```code

test dataset:
id,anchor,target,context
4112d61851461f60,opc drum,inorganic photoconductor drum,G02
09e418c93a776564,adjust gas flow,altering gas flow,F23
36baf228038e314b,lower trunnion,lower locating,B60
1f37ead645e7f0c8,cap component,upper portion,D06
71a5b6ad068d531f,neural stimulation,artificial neural network,H04
474c874d0c07bd21,dry corn,dry corn starch,C12
```


Key Idea:
If the test set contains anchors (specific keywords or reference terms) that never appear in the training set, the model must effectively generalize to entirely new linguistic inputs it has never seen before. On the other hand, if during cross-validation (CV) you do not mimic this scenario—i.e., if you allow the same anchors to appear in both training and validation folds—your validation performance may be overly optimistic. This happens because the model can “memorize” or become finely tuned to those known anchors, thus inflating the validation scores.

Detailed Explanation:

What are anchors?
In this dataset, an "anchor" is a key term or entity around which a particular context or target meaning is provided. For example, "abatement" is an anchor. The model learns how the anchor relates to various targets and contexts.

When the test set has unseen anchors:
Suppose the test set includes an anchor that never occurs in the training data. During inference, the model will see this new anchor and must rely on a generalized understanding of language or semantic relationships to predict the correct score. There is no direct anchor-specific pattern the model can exploit, forcing it to extrapolate beyond its known training examples.

If CV does not respect this distribution:
Let’s say in your cross-validation splits, you mix all anchors so that each fold contains a similar distribution of anchors. In that situation:

Each validation fold ends up containing anchors that the model has already “seen” in training.
The model’s performance on validation will look better because it’s already familiar with those exact anchors. It can recall anchor-specific associations without having to genuinely generalize.
This inflated performance does not reflect the real challenge posed by unseen anchors in the test set.
Resulting Mismatch (Overfitting to Known Anchors):
Because your model validation does not mirror the actual test scenario (where new anchors appear), it can inadvertently overfit to the known anchors. This results in:

Strong CV performance (since validation sets contain known anchors).
Weaker performance on the leaderboard (LB) or test set (which has unknown anchors).
The mismatch emerges because the model was never evaluated on completely new anchors during validation, so you didn’t catch how well it generalizes. It "overfits" by becoming too dependent on patterns seen in the training anchors.